In [ ]:
Импорт библиотек, не связанных с PySpark 

In [1]:
# The Jupyter Notebook and dataset were downloaded from https://www.kaggle.com/code/tylerx/machine-learning-with-spark

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['.ipynb_checkpoints', 'airports.csv', 'clearDF.csv', 'flights.csv', 'Lab2df.csv', 'raw-flight-data.csv']


In [ ]:
Импорт PySpark библиотек

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler

from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
Загрузка данных

In [3]:
csv = spark.read.csv('./input/clearDF.csv', inferSchema=True, header=True)
csv.show(10)

+-------------------+-------------------+------------------+-------------------+------------------+----------+
|      energy_median|        energy_mean|        energy_max|         energy_std|        energy_sum|energy_min|
+-------------------+-------------------+------------------+-------------------+------------------+----------+
|             0.1415|0.29616666875000003|         1.1160001| 0.2814713178628203|14.216000100000002|     0.031|
|             0.1015|          0.1898125|             0.685| 0.1884046862418033|             9.111|     0.064|
|              0.114| 0.2189791666666666|0.6759999999999999|0.20291927853038208|10.510999999999996|     0.065|
|              0.191|0.32597916666666665|0.7879999999999999| 0.2592049619947409|15.646999999999998|     0.066|
|0.21800000000000005|             0.3575|             1.077|0.28759657027517305|             17.16|     0.066|
|             0.1305| 0.2350833333333333|             0.705| 0.2220696491599295|            11.284|     0.066|
|

In [ ]:
Создание бинарного признака

In [4]:
mean_std = csv.select(mean('energy_std')).collect()[0][0]
data = csv.select("energy_median", "energy_mean","energy_std", "energy_max", "energy_sum", "energy_min", ((col("energy_std") > mean_std).cast("Int").alias("binary_std")))
data.show(10)

+-------------------+-------------------+-------------------+------------------+------------------+----------+----------+
|      energy_median|        energy_mean|         energy_std|        energy_max|        energy_sum|energy_min|binary_std|
+-------------------+-------------------+-------------------+------------------+------------------+----------+----------+
|             0.1415|0.29616666875000003| 0.2814713178628203|         1.1160001|14.216000100000002|     0.031|         1|
|             0.1015|          0.1898125| 0.1884046862418033|             0.685|             9.111|     0.064|         1|
|              0.114| 0.2189791666666666|0.20291927853038208|0.6759999999999999|10.510999999999996|     0.065|         1|
|              0.191|0.32597916666666665| 0.2592049619947409|0.7879999999999999|15.646999999999998|     0.066|         1|
|0.21800000000000005|             0.3575|0.28759657027517305|             1.077|             17.16|     0.066|         1|
|             0.1305| 0.

In [ ]:
Разделение на обучающую и тестовую выборки

In [5]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Обучающая:", train_rows, " Тестирующая:", test_rows)

Обучающая: 2137043  Тестирующая: 916209


In [ ]:
Разделение на обучающую и тестовую выборки

In [6]:
numVect = VectorAssembler(inputCols = ["energy_median", "energy_mean", "energy_max", "energy_sum", "energy_min"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="features")

regresion = LinearRegression(labelCol="energy_std",featuresCol="features",maxIter=2,regParam=0.8, elasticNetParam=0.0)
regressionPipeline = Pipeline(stages=[numVect, minMax, regresion])

classifier = RandomForestClassifier(labelCol="binary_std",featuresCol="features", maxDepth=2, maxBins=5, numTrees=5)
classifierPipeline = Pipeline(stages=[numVect, minMax, classifier])

In [7]:
regressionPiplineModel = regressionPipeline.fit(train)

In [8]:
classifierPiplineModel = classifierPipeline.fit(train)

In [ ]:
Вывод векторных предсказаний

In [9]:
print("Матрица для линейной регрессии:")
regressionPrediction = regressionPiplineModel.transform(test)
regressionPredicted = regressionPrediction.select("features", "prediction", "energy_std")
regressionPredicted.show(1000, truncate=False)

print("Матрица для случайного леса:")
classifierPrediction = classifierPiplineModel.transform(test)
classifierPredicted = classifierPrediction.select("features", "prediction", "binary_std")
classifierPredicted.show(1000, truncate=False)

Матрица для линейной регрессии:
+-----------------------------------------------------------------------------------------------------------+-------------------+---------------------+
|features                                                                                                   |prediction         |energy_std           |
+-----------------------------------------------------------------------------------------------------------+-------------------+---------------------+
|(5,[],[])                                                                                                  |0.09637123479177558|0.0                  |
|(5,[],[])                                                                                                  |0.09637123479177558|0.0                  |
|(5,[],[])                                                                                                  |0.09637123479177558|0.0                  |
|(5,[],[])                                              

In [ ]:
Метрики R2 и RMSE для линейной регрессии

In [10]:
regressionEvaluatorRMSE = RegressionEvaluator(labelCol="energy_std", predictionCol = "prediction", metricName="rmse")
rmse = regressionEvaluatorRMSE.evaluate(regressionPrediction)
print("RMSE(Среднеквадратичная ошибка) = %g" % rmse)

regressionEvaluatorR2 = RegressionEvaluator(labelCol="energy_std", predictionCol = "prediction", metricName="r2")
r2 = regressionEvaluatorR2.evaluate(regressionPrediction)
print("R2(Коэффициент детерминации) = %g" % r2)

RMSE(Среднеквадратичная ошибка) = 0.0773192
R2(Коэффициент детерминации) = 0.376467


In [ ]:
Построение матрицы ошибок для случайного леса и вывод метрики AUR(Area Under ROC)

In [11]:
def MetricsCalculations(predicted):
    tp = float(predicted.filter("prediction == 1.0 AND binary_std= 1").count())
    fp = float(predicted.filter("prediction == 1.0 AND binary_std= 0").count())
    tn = float(predicted.filter("prediction == 0.0 AND binary_std= 0").count())
    fn = float(predicted.filter("prediction == 0.0 AND binary_std= 1").count())
    
    pr = tp / (tp + fp)
    re = tp / (tp + fn)
    metrics = spark.createDataFrame([
     ("TP", tp),
     ("FP", fp),
     ("TN", tn),
     ("FN", fn),
     ("Precision", pr),
     ("Recall", re),
     ("F1", 2*pr*re/(re+pr))],["metric", "value"])
    metrics.show()
print("Матрица ошибок для случайного леса:")
MetricsCalculations(classifierPredicted)    

classifierEvaluatorAUR = BinaryClassificationEvaluator(labelCol = "binary_std", rawPredictionCol = "rawPrediction", metricName = "areaUnderROC")
aur = classifierEvaluatorAUR.evaluate(classifierPrediction)
print ("AUR(Площадь под рабочей характеристикой приемника)= %g" % aur)

Матрица ошибок для случайного леса:
+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|          345949.0|
|       FP|           31727.0|
|       TN|          499602.0|
|       FN|           38931.0|
|Precision|0.9159941325368834|
|   Recall|0.8988489918935773|
|       F1|0.9073405756429691|
+---------+------------------+

AUR(Площадь под рабочей характеристикой приемника)= 0.956175


In [ ]:
Валидация модели регрессии

In [12]:
regressionGrid = ParamGridBuilder()

regressionGrid = regressionGrid.addGrid(regresion.maxIter, [10, 20, 30])
regressionGrid = regressionGrid.addGrid(regresion.regParam, [0.1, 0.2, 0.3])
regressionGrid = regressionGrid.addGrid(regresion.elasticNetParam, [0.2, 0.5, 0.8])

regressionGrid = regressionGrid.build()

regressionValidation = CrossValidator(estimator = regressionPipeline, 
                     evaluator = regressionEvaluatorR2,
                     estimatorParamMaps = regressionGrid,
                     numFolds = 2)

regressionNewModel = regressionValidation.fit(train)

In [ ]:
Вывод метрик R2 и RMSE для новой линейной регрессии

In [13]:
regressionNewPrediction = regressionNewModel.transform(test)

regressionBestModel = regressionNewModel.bestModel.stages[2] 
print("Модель линейной регрессии с параметрами MaxIter = %s, RegParam = %s, ElasticNet = %s" %
     (regressionBestModel.getMaxIter(),
      regressionBestModel.getRegParam(),
      regressionBestModel.getElasticNetParam()))

regressionNewEvaluatorRMSE = RegressionEvaluator(labelCol="energy_std", predictionCol = "prediction", metricName="rmse")
rmse = regressionNewEvaluatorRMSE.evaluate(regressionNewPrediction)
print("RMSE(Среднеквадратичная ошибка) = %g" % rmse)

regressionNewEvaluatorR2 = RegressionEvaluator(labelCol="energy_std", predictionCol = "prediction", metricName="r2")
r2 = regressionNewEvaluatorR2.evaluate(regressionNewPrediction)
print("R2(Коэффициент детерминации) = %g" % r2)

Модель линейной регрессии с параметрами MaxIter = 10, RegParam = 0.1, ElasticNet = 0.2
RMSE(Среднеквадратичная ошибка) = 0.0515972
R2(Коэффициент детерминации) = 0.722325


In [ ]:
Валидация модели классификации

In [14]:
classifierGrid = ParamGridBuilder()

classifierGrid = classifierGrid.addGrid(classifier.maxDepth, [2, 10, 5])
classifierGrid = classifierGrid.addGrid(classifier.maxBins, [5, 10, 20])
classifierGrid = classifierGrid.addGrid(classifier.numTrees, [5, 20, 50])

classifierGrid = classifierGrid.build()

classifierValidation = CrossValidator(estimator = classifierPipeline, 
                                      evaluator = classifierEvaluatorAUR, 
                                      estimatorParamMaps = classifierGrid, 
                                      numFolds = 2)

classifierNewModel = classifierValidation.fit(train)

In [ ]:
Построение новой матрицы ошибок для случайного леса и вывод метрики AUR(Area Under ROC)

In [15]:
classifierNewPrediction = classifierNewModel.transform(test)

classifierBestModel = classifierNewModel.bestModel.stages[2] 
print("Модель случайного леса с параметрами MaxDepth = %s, MaxBins = %s, NumTrees = %s" % 
     (classifierBestModel.getMaxDepth(),
      classifierBestModel.getMaxBins(),
      classifierBestModel.getNumTrees))

MetricsCalculations(classifierNewPrediction)

classifierNewEvaluator = BinaryClassificationEvaluator(labelCol="binary_std", rawPredictionCol = "rawPrediction", metricName="areaUnderROC")
aur = classifierNewEvaluator.evaluate(classifierNewPrediction)
print ("AUR(Площадь под рабочей характеристикой приемника)= %g" % aur)

Модель случайного леса с параметрами MaxDepth = 10, MaxBins = 20, NumTrees = 50
+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|          367206.0|
|       FP|           18547.0|
|       TN|          512782.0|
|       FN|           17674.0|
|Precision|0.9519200109914894|
|   Recall|0.9540791935148618|
|       F1|0.9529983792544571|
+---------+------------------+

AUR(Площадь под рабочей характеристикой приемника)= 0.994462
